In [16]:
import json
import pandas as pd
import urllib.parse
from collections import Counter
import numpy as np

In [17]:
### PARAMETERS
# -------------

jsonPath = "C:/Users/agrog/Documents/Oxford/analysis/pointer/alpha/data/jan24Pilot/pointer24.json"
csvPath = "C:/Users/agrog/Documents/Oxford/analysis/pointer/alpha/data/jan24Pilot/pointerPilotRounds.csv"
# -------------

In [18]:
with open(jsonPath, 'rb') as h:
    qs = json.load(h)

# Take just the fields component
data = [i['fields'] for i in qs]

In [19]:
dataDf = pd.DataFrame.from_dict(data)

In [20]:
df = dataDf.loc[(dataDf["Source"] == "oxford24") & (~dataDf["UserId"].isin(["772", "test", "805873905", "1012720441", "1389524335", "114796789"]))]

In [29]:
df.head()

,UserId,RawData,Source,IP,platformData,Metadata,timeCreated,lastModified
83,1359218913,"[[], [], [{'resp': {'action': ['click_3'], 'al...",oxford24,,{},"{'id': '1359218913', 'iv': None, 'tag': None, ...",2023-12-10T04:19:33.273Z,2023-12-11T19:23:11.146Z
86,1194787547,"[[{'resp': {'action': ['click_3', 'click_end',...",oxford24,,"[ f, m, ,]","{'id': '1194787547', 'iv': None, 'tag': None, ...",2023-12-18T16:12:42.503Z,2023-12-18T16:16:19.358Z
87,777757392,"[[{'resp': {'action': ['click_2', 'click_3', '...",oxford24,,[they do math operations on the current stock ...,"{'id': '777757392', 'iv': None, 'tag': None, '...",2024-01-18T17:56:18.416Z,2024-01-19T11:49:15.505Z
88,683539231,"[[{'resp': {'action': ['click_3', 'click_start...",oxford24,,"[Some had set values, others had values that w...","{'id': '683539231', 'iv': None, 'tag': None, '...",2024-01-19T10:50:58.680Z,2024-01-19T11:15:59.194Z
92,1102458484,"[[{'resp': {'action': ['click_3'], 'allowed': ...",oxford24,,[all images corresponded to a mathematical ope...,"{'id': '1102458484', 'iv': None, 'tag': None, ...",2024-01-22T15:05:40.409Z,2024-01-22T15:52:40.579Z


In [72]:
data[0]['RawData']

[{'resp': {'action': ['click_3', 'click_start', 'click_2', 'click_start'],
   'allowed': [True, False, False, False],
   'timestamp': [1697122023622, 1697122028508, 1697122029492, 1697122031773]},
  'level': 1,
  'score': -2,
  'layout': {'id': 547,
   'nodes': [{'id': 'start',
     'level': '1',
     'concept': 'start',
     'connections': ['2', '3']},
    {'id': '2', 'level': '2', 'concept': 'A', 'connections': ['1', 'end']},
    {'id': '3', 'level': '2', 'concept': 'C', 'connections': ['1', 'end']},
    {'id': 'end',
     'level': '999',
     'concept': 'end',
     'connections': ['2', '3']}],
   'scores': {'all': [{'path': ['1', '2', '4'], 'score': 0},
     {'path': ['1', '3', '4'], 'score': -2}],
    'goal': -2,
    'start': -1},
   'endIndex': '4',
   'givenName': 'level_1_47',
   'curriculumLevel': 9,
   'difficultyLevel': 1},
  'target': -2,
  'endTime': 1697122033788,
  'layoutId': 'level_1_47',
  'playerIx': '3',
  'completed': True,
  'roundType': 'train',
  'startTime': 169

In [53]:
# Our base dict that will be attached to all round-level data
singleDatum = {'userId' : data[0]['UserId'], 'demographics':data[0]['platformData'], 'source' : data[0]['Source']}

In [45]:
singleDatum

{'userId': 'kiLeRcUO5JsV5Na7lcP8kvgnVIcR87ZleRoAlox32hEvjFmefAIvIhnRKYcZkyslSPKXpAlkX7nkZfZjWGgqvUAvAys3h1mU2HMsUxEAbyCn3V7Ld3XFSwp3JdYWBUJr',
 'demographics': {'age': '21-30',
  'gender': 'male',
  'prolificId': '6312f83ee5b1e762f3648060'},
 'source': 'prolific'}

In [40]:
def getRounds(df, save=False):
    # Take in a pre-filtered dataframe and return a dataframe of rounds, and save if specified
    
    # Convert back to a dict to operate on
    data = pd.DataFrame.to_dict(df, orient="index")
    # Iterate over each participant
    roundLevelData = []
    for user in data:
        # Define a dict with information common across rounds (eg. userId etc.)
        common = {'userId' : data[user]['UserId'], 'source' : data[user]['Source']}
        # In pointer the player can come back multiple times, so we get a nested list
        # Create a list of attempt times
        flatList = np.asarray(data[user]['RawData'], dtype=object).flatten()
        for ix, attempt in enumerate(data[user]['RawData']):
            if len(attempt) > 0:
                for r in attempt:
                    try:
                        tmp = {**common, **r, **{"gameNum": ix}, **{"curriculumType" : data[user]['Metadata']["curriculumType"], "gameComplete" : (attempt[-1]["roundIndex"] >= 93)}}
                    except:
                        tmp = {**common, **r, **{"gameNum": ix, "gameComplete" : (attempt[-1]["roundIndex"] >= 93)}}
                    roundLevelData.append(tmp)
    return roundLevelData
    
    

In [41]:
rld = getRounds(df)
rldf = pd.DataFrame.from_dict(rld)
# rldf.to_csv(csvPath, index=False)